In [2]:
# Download files from HDFS into working environment
! [ -d monte-carlo-risk ] || (echo "Downloading prepared data from HDFS. Please wait..." ; \
                    hdfs dfs -copyToLocal /financial_risk . ; echo "Done!";)

2024-12-05 11:30:23,780 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Done!


In [2]:
# Create function to parse downloaded data that doesn't come from yahoo finance
from datetime import datetime
from datetime import timedelta
from itertools import islice
%matplotlib inline
import numpy as np
import statsmodels.api as sm
import warnings
warnings.filterwarnings('ignore')

plt.rcParams['figure.figsize'] = (15,7)

#base_folder = "financial_risk/"
#factors_folder= base_folder + "factors/"

factors_folder = "financial_risk/"



# read data from local disk
def readInvestingDotComHistory(fname):
    def process_line(line):
        cols = line.split(",")
        date = datetime.strptime(cols[0], "%b %d, %Y")
        value = float(cols[1])
        return (date, value)
    
    with open(fname) as f:
        content_w_header = f.readlines()
        # remove the first line 
        # and reverse lines to sort the data by date, in ascending order
        content = content_w_header[:0:-1]
        return list(map(process_line , content))

factor1_files = ['crudeoil.csv', 'us30yeartreasurybonds.csv']
factor1_files = map(lambda fn: factors_folder + fn, factor1_files)
factors1 = [readInvestingDotComHistory(f) for f in factor1_files]

print(factors1[0][:5])

ModuleNotFoundError: No module named 'matplotlib'